# Econ 8210 Quant Macro, Homework 1
## Part 2 - Solution Methods
Haosi Shen, Fall 2024

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import time
import torch

np.random.seed(42) 

## Computing Pareto Efficient Allocations

Consider an endowment economy with $m$ different goods and $n$ agents. Each agent $i = 1, ..., n$ has an endowment $e_j^i >0$ for every $j = 1, ..., m$ and a utility function of the form
$$ u^i (x) = \sum_{j=1}^{m} \alpha_j \frac{x_{j}^{1+\omega_j^i}}{1+\omega_j^i} $$
where $\alpha_j > 0 > \omega_j^i$ are agent-specific parameters.

Given some social weights $\lambda_i > 0$, solve for the social planner’s problem for $m = n = 3$ using the **Adam (Adaptive Moment Estimation)** method. Try different values of $\alpha_j,\;\omega_j^i,\;\lambda_i$. 

$$ \max_{\{x^i\}} \; \sum_{i=1}^{n} \lambda_i u^{i}(x)$$

Compute first the case where all the agents have the same parameters and
social weights and later a case where there is a fair degree of heterogeneity.

How does the method perform? How does heterogeneity in the agent-specific parameters
affect the results?

Can you handle the case where $m = n = 10$?

> I choose to use **Adam** since it is more efficient for high-dimensional optimization problems and offers more stability and robustness. However, if we are only solving for the case of $m=n=3$, then the Newton-Raphson method might be more ideal since this problem is relatively smooth. Adam only requires gradient information and does not involve inverting the Hessian.

In [2]:
# utility function for agent i
def utility(x, alpha, omega):
    return torch.sum(alpha * (x ** (1 + omega)) / (1 + omega))


# Incorporate resource constraints for each good j
# Define total endowments
endowments = torch.tensor([30.0, 30.0, 30.0], dtype = torch.float32)

### Case I: Homogeneous Agents

> $m = n = 3$

All agents $j$ have the same parameters $\alpha_j, \omega_j^i$ and social weights $\lambda_j$. 

In [3]:
# Social planner's objective, with Homogeneous Agents
def social_planner_objective_homog(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j, omega_j)
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative

In [4]:
import torch.optim as optim

# =============== DEFINE PARAMETERS ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32) 
omega_j = torch.tensor([[0.5, 0.5, 0.5]] * n, dtype = torch.float32)
lambda_i = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32)  # Pareto weights

# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_homog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)


Iteration 99: Objective = 41.48536682128906
Iteration 199: Objective = 89.36106872558594
Iteration 299: Objective = 157.54916381835938
Iteration 399: Objective = 239.6324462890625
Iteration 499: Objective = 331.9831237792969
Iteration 599: Objective = 432.8163146972656
Iteration 699: Objective = 541.0831909179688
Iteration 799: Objective = 569.2794799804688
Iteration 899: Objective = 569.2852172851562
Iteration 999: Objective = 569.2862548828125

 Optimal allocations for Homogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.940836  10.041025  10.073340
Agent 2  10.049340  10.083452   9.971957
Agent 3  10.014572   9.880261   9.959444


### Case II: Heterogeneous Agents 
> $m = n = 3$

* Each agent $i$ has their own set of $\alpha_j, \omega_j^i$ parameters.
* Pareto weights $\lambda_i$ differ among agents.
* Resource constraint remains the same. Total endowment for each good is 30.

In [5]:
# Social planner's objective, with Heterogeneous Agents

def social_planner_objective_heterog(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j[i], omega_j[i])
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative


In [6]:
# =============== DEFINE PARAMETERS ===============
alpha_j = torch.tensor([[1.0, 0.8, 1.2],  # agent 1
                        [1.1, 0.9, 1.3],  # agent 2
                        [0.9, 0.7, 1.1]])  # agent 3

omega_j = torch.tensor([[0.3, 0.5, 0.7], 
                        [0.4, 0.6, 0.8],  
                        [0.5, 0.4, 0.6]]) 

lambda_i = torch.tensor([0.9, 1.1, 1.0])  # Social weights


# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)
#x_i = (torch.rand((n, m), requires_grad=True) * endowments / n).clone().detach().requires_grad_(True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_heterog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 14.429256439208984
Iteration 1999: Objective = 31.05494499206543
Iteration 2999: Objective = 53.16339111328125
Iteration 3999: Objective = 78.79014587402344
Iteration 4999: Objective = 107.16829681396484
Iteration 5999: Objective = 138.00628662109375
Iteration 6999: Objective = 171.1595458984375
Iteration 7999: Objective = 206.53012084960938
Iteration 8999: Objective = 219.7765655517578
Iteration 9999: Objective = 225.86273193359375

 Optimal allocations for Heterogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.081430   9.787652  10.099073
Agent 2  10.296638  11.073445  11.160144
Agent 3  10.623183   9.140093   8.743629


> The `Adam` optimizer performs fairly well for both the homogeneous and heterogeneous agents cases. The computations are fast and produce sufficiently accurate results. 
> 
> Introducing heterogeneity in the agent-specific parameters does affect the socially optimal allocations, and significantly slow down convergence of optimization.

### Case III: 10 Agents, 10 Goods

> $m=n=10$

Since the `Adam` optimizer generally works well for higher-dimension problems, we now try computing the Pareto efficient allocations of a 10-agent 10-good economy. 

**Consider homogeneity across agents.**

In [17]:
m, n = 10, 10  # 10 goods, 10 agents
endowments = torch.tensor([30.0] * m)  # total endowments for 10 goods

alpha_j = torch.tensor([1.0] * m)  
omega_j = torch.tensor([0.5] * m)  
lambda_i = torch.tensor([1.0] * n)

x_i = torch.rand((n, m), requires_grad = True)  # initial allocations

optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_homog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)

Iteration 99: Objective = 157.95803833007812
Iteration 199: Objective = 325.2500915527344
Iteration 299: Objective = 345.5758361816406
Iteration 399: Objective = 346.7998962402344
Iteration 499: Objective = 346.927734375
Iteration 599: Objective = 346.99945068359375
Iteration 699: Objective = 346.41583251953125
Iteration 799: Objective = 347.06060791015625
Iteration 899: Objective = 346.8817138671875
Iteration 999: Objective = 346.8741149902344

 Optimal allocations for Homogeneous Agents:
            Good 1    Good 2    Good 3    Good 4    Good 5    Good 6  \
Agent 1   2.852486  3.065950  2.721496  3.140219  2.844044  3.327388   
Agent 2   3.098233  2.968297  2.921544  2.665549  2.999330  2.875577   
Agent 3   2.542581  3.226127  2.840879  3.262613  3.279150  2.779028   
Agent 4   3.382443  2.729536  3.246885  2.885556  2.965294  3.184775   
Agent 5   3.015020  3.292438  3.246809  3.255260  2.715430  2.876262   
Agent 6   2.726313  2.929272  2.927523  2.957928  3.173136  2.718218   
A

**Now, try the heterogeneous agents case.**

In [29]:
# Heterogeneous agent-specific parameters
alpha_j = torch.tensor([
    [1.0, 0.8, 1.2, 0.9, 1.1, 0.7, 1.3, 0.95, 1.05, 1.15] for _ in range(n)]) 

omega_j = torch.tensor([
    [0.3, 0.4, 0.5, 0.35, 0.45, 0.55, 0.25, 0.6, 0.65, 0.5] for _ in range(n)])

lambda_i = torch.tensor([0.9, 1.1, 0.8, 1.2, 1.0, 0.85, 1.15, 0.95, 1.05, 1.0])


In [38]:
x_i = torch.rand((n, m), requires_grad=True)
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_heterog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 168.5098876953125
Iteration 1999: Objective = 333.4227294921875
Iteration 2999: Objective = 350.2723388671875
Iteration 3999: Objective = 360.41552734375
Iteration 4999: Objective = 384.34906005859375
Iteration 5999: Objective = 425.4910888671875
Iteration 6999: Objective = 477.1968994140625
Iteration 7999: Objective = 530.7722778320312
Iteration 8999: Objective = 582.2130126953125
Iteration 9999: Objective = 628.9527587890625

 Optimal allocations for Heterogeneous Agents:
            Good 1    Good 2    Good 3    Good 4     Good 5    Good 6  \
Agent 1   0.090776  0.170010  0.048248  0.128603   0.107813  0.091446   
Agent 2   7.479658  6.854785  8.812072  8.476297   8.428061  8.622780   
Agent 3   0.093248  0.119695  0.054456  0.099403   0.077333  0.084708   
Agent 4   9.115497  9.180102  9.731377  9.565689   9.611230  9.415743   
Agent 5   1.066900  0.748357  0.208702  0.244601   0.827350  1.838966   
Agent 6   0.072845  0.089515  0.038561  0.145964   0.096

> The method works well for $m=n=10$, even after we introduce household heterogeneity. 

## Computing Equilibrium Allocations

Using the same model as in the previous exercise. Find the equilibrium prices for each good $p^j$.

1. Solve for the first-order conditions of each agent.
1. Aggregate the excess demands. 
1. Solve the resulting system of nonlinear equations, when all markets clear.

### Homogeneous Agents, $m=n=3$

**FOCs**
> To find the demand for each good, we solve the Lagrangean:
> $$\mathcal{L}=\sum_{j=1}^{m}\alpha_j \cdot \frac{x_{ij}^{1+\omega}}{1+\omega}-\lambda_i \Big(\sum_{j=1}^{m}p_j\cdot x_{ij} - Y_i\Big)$$
> where $Y_i$ is the income of agent $i$. 
> The FOCs for this problem yield
> $$ x_{ij} = \frac{\alpha_j \cdot p_{j}^{-\omega}}{\sum_{k=1}^{m} \alpha_k \cdot p_{k}^{-\omega} }\cdot Y_i $$

**Excess Demand**
> Aggregate the demand across agents and compare it to the total endowment to find the excess demand for each good
> $$z_j(p) =  \sum_{i=1}^{n} x_{ij}(p) - e_j$$
> Markets clear when $z_j(p)=0$ for all goods $j$.

### Heterogeneous Agents, $m=n=3$
